## Setup

This notebook is used to run the vial swap error detection pipeline on drug drawup videos.

***The code needs to be executed in a GPU runtime. From the menu bar at the top of the screen, select Runtime > Change runtime type > T4 GPU (or other GPU if this is not available).***

To run the code in each cell of this notebook, highlight your mouse over the cell, then click the Play button on the left.

This will execute the command in the cell.

The command output will be printed below each cell.

When the command finishes, a green checkmark will be displayed to the left of the cell.

All data resides in the notebook, and all code is executed within it. No data needs to be downloaded to your local computer.


In [ ]:
# 1) Retrieve example videos:
# a) Video depicting correct drug drawup in real-world operating room scenario, from a propofol syringe to a propofol vial
# b) Video depicting a vial swap error demonstrated by trained researcher in controlled environment, from a midazolam syringe to a dexamethasone vial

!gdown 1yAprqI1XQD7gNNqDD80-O_GuKVed_dHW
!gdown 1pcU01QPMa2HXNODmHGLmOsffLqcs7QZ5

# If you want to view the videos locally,
# Click the Folder icon on the left > Click the Refresh icon >
# Click the three vertical dots next to the desired video > Click Download

# 2) Download and unzip project code
!gdown 1uHYxyHIK6k0p2LW_8ade6XM2zKApg_jA # files
!unzip -q code.zip -d .

# 3) Download weights necessary for the detector and classifier. Runtime: 97 seconds
!gdown 19pns6_7PHnEbqM1C46uJuUDmd45d6NUy # Syringe detector weights
!gdown 1Fzxvnk4PbTjcIqAOPjG4k6Vc_5z6Htrz # Vial detector weights
!gdown 14yKlH7nMy3mN4r34KhmtoHRc9QmKUY6f # Syringe classifier weights
!gdown 1XE6tPwbAY3AqwwyLNP7RXUjwn_8HdofB # Vial classifier weights

# 4) Install CLIP network dependency for classification. Runtime: 89 seconds
!pip install git+https://github.com/openai/CLIP.git@40f5484c1c74edd83cb9cf687c6ab92b28d8b656

## Detector

In this section, we run the detector over each recorded video.
For each video, we run an object detector for the syringe label, and an object detector for the vial label. The command will output the progress as follows:

```
detect: weights=['/content/syringe_detector_weights.pt'], source=/content/vid1.mp4, data=code/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.8, iou_thres=0.45, max_det=1, device=, view_img=False, save_txt=True, save_conf=True, save_crop=True, nosave=False, classes=[1], agnostic_nms=False, augment=True, visualize=False, update=False, project=/content/syringe_frames/, name=vid1, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
requirements: /content/code/requirements.txt not found, check failed.
YOLOv5 🚀 2024-4-12 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers...
Model summary: 444 layers, 86180143 parameters, 0 gradients
video 1/1 (1/1689) /content/vid1.mp4: 640x384 Done. (0.255s)
video 1/1 (2/1689) /content/vid1.mp4: 640x384 Done. (0.141s)
video 1/1 (3/1689) /content/vid1.mp4: 640x384 Done. (0.096s)
video 1/1 (4/1689) /content/vid1.mp4: 640x384 Done. (0.088s)
video 1/1 (5/1689) /content/vid1.mp4: 640x384 Done. (0.082s)
video 1/1 (6/1689) /content/vid1.mp4: 640x384 Done. (0.082s)
```

In [ ]:
# Run object detector on video 1
# Running time is 3 minutes 30 seconds for each of the 2 commands
!python /content/code/detect.py --augment --max-det 1 --classes 1 --project /content/syringe_frames/ --name vid1 --conf-thres 0.8 --save-txt --save-crop --save-conf --weights /content/syringe_detector_weights.pt --img 640 --source /content/vid1.mp4
!python /content/code/detect.py --augment --max-det 1 --classes 1 --project /content/vial_frames/ --name vid1 --conf-thres 0.8 --save-txt --save-crop --save-conf --weights /content/vial_detector_weights.pt --img 640 --source /content/vid1.mp4

In [ ]:
# Run object detector on video 2
# Running time is 50 seconds for each of the 2 commands
!python /content/code/detect.py --augment --max-det 1 --classes 1 --project /content/syringe_frames/ --name vid2 --conf-thres 0.8 --save-txt --save-crop --save-conf --weights /content/syringe_detector_weights.pt --img 640 --source /content/vid2.mp4
!python /content/code/detect.py --augment --max-det 1 --classes 1 --project /content/vial_frames/ --name vid2 --conf-thres 0.8 --save-txt --save-crop --save-conf --weights /content/vial_detector_weights.pt --img 640 --source /content/vid2.mp4

## Classifier

Here, we run the syringe label and vial label classifier across each frame and record the frame-level outputs to a log file.

The expected output for each command in this step is below:
```
Loading model ...
Finished loading model
Loading weights ...
Finished loading weights
23% 5/22 [00:10<00:29,  1.71s/it]

```
Ignore any runtime warnings.

In [ ]:
# Run syringe label classifier on frames of video 1. Runtime: 1 minute 9 seconds
!python /content/code/Inference_vid.py --obj 'syringe' --exp "vid1" --weights 'syringe_classifier_weights.pt' --batch-size 32 --model 'ViT-L/14'
# Run vial label classifier on frames of video 1. Runtime: 1 minute 1 seconds
!python /content/code/Inference_vid.py --obj 'vial' --exp "vid1" --weights 'vial_classifier_weights.pt' --batch-size 32 --model 'ViT-L/14'

In [ ]:
# Run syringe label classifier on frames of video 2. Runtime: 50 seconds
!python /content/code/Inference_vid.py --obj 'syringe' --exp "vid2" --weights 'syringe_classifier_weights.pt' --batch-size 32 --model 'ViT-L/14'
# Run vial label classifier on frames of video 2. Runtime: 50 seconds
!python /content/code/Inference_vid.py --obj 'vial' --exp "vid2" --weights 'vial_classifier_weights.pt' --batch-size 32 --model 'ViT-L/14'

Finally, we parse the log file and output the video-level classification.
The expected output from this step is:
```
vid1
Syringe: Propofol
Vial: Propofol
No error
vid2
Syringe: Midazolam
Vial: Dexamethasone
Vial swap error
```

In [ ]:
# Compute video-level classification result for video 1. Runtime: 0.4 seconds
!python /content/code/classify.py --exp vid1
# Compute video-level classification result for video 2. Runtime: 0.4 seconds
!python /content/code/classify.py --exp vid2